# Backtesting Mutant v23.08

## Backtest Method
    30 days as a group, randomly choose 30 continuos 30 days to see the performance

In [1]:
# from __future__ import (absolute_import, division, print_function,
#                         unicode_literals)
import datetime
# import os.path
import sys

import pandas as pd
import numpy as np
import backtrader as bt
import backtrader.indicators as ta

import matplotlib
import tkinter
matplotlib.use('TKAgg')
# matplotlib.use('QT5Agg')

from mutant.model import Mutant
from mutant.strategy import MutantBacktrader

raw_data_path = "../data/BTCUSD_latest.csv"
# raw_data_path = "../data/Raw_BTCUSDT1708-2303.csv"

## Load data

In [2]:
dataframe = pd.read_csv(raw_data_path,
                                parse_dates=True,
                                index_col=0)
dataframe.index = pd.to_datetime(dataframe.index, format='ISO8601')

In [3]:
model = Mutant()
trade_reports = []
drawdown_reports = []
total_sessions = 1
for i in range(total_sessions):
    backtest_length = 1440*30
    start = np.random.choice(len(dataframe))
    end = start + backtest_length
    df = dataframe.iloc[start:end]
    df = df.groupby(pd.Grouper(freq='5Min')).agg({"open": "first", 
                                                  "high": "max",
                                                  "low": "min",
                                                  "close": "last",
                                                  "volume": "sum"})
    data = bt.feeds.PandasData(
        dataname=df,
        # fromdate=datetime.datetime(2022, 9, 1),
        # todate=datetime.datetime(2022, 9, 2),
        datetime=None,)
    cerebro = bt.Cerebro()
    cerebro.addstrategy(MutantBacktrader, model)
    cerebro.adddata(data)
    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='mutant_trade')
    cerebro.addanalyzer(bt.analyzers.DrawDown, _name='mutant_drawdown')
    cerebro.addsizer(bt.sizers.PercentSizer, percents=10)
    cerebro.broker.setcash(100000.0)
    cerebro.broker.setcommission(commission=0.0004)
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
    results = cerebro.run()
    result = results[0]
    trade_reports.append(result.analyzers.mutant_trade.get_analysis())
    drawdown_reports.append(result.analyzers.mutant_drawdown.get_analysis())
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 100000.00
2018-07-16T03:20:00, BUY CREATE, 6361.02
2018-07-16T03:25:00, BUY EXECUTED, Price: 6361.02, Cost: 10000.00, Comm 4.00
2018-07-16T08:50:00, SELL EXECUTED, Price: 6402.17, Cost: 10000.00, Comm 4.03
2018-07-16T08:50:00, Order Canceled/Margin/Rejected
2018-07-16T08:50:00, OPERATION PROFIT, GROSS 64.70, NET 56.67
2018-07-17T11:55:00, SELL CREATE, 6675.76
2018-07-17T12:00:00, SELL EXECUTED, Price: 6675.76, Cost: -10005.67, Comm 4.00
2018-07-17T15:55:00, BUY EXECUTED, Price: 6788.09, Cost: -10005.67, Comm 4.07
2018-07-17T15:55:00, Order Canceled/Margin/Rejected
2018-07-17T15:55:00, OPERATION PROFIT, GROSS -168.35, NET -176.43
2018-07-20T12:40:00, SELL CREATE, 7445.86
2018-07-20T12:45:00, SELL EXECUTED, Price: 7447.00, Cost: -9989.55, Comm 4.00
2018-07-20T16:35:00, BUY EXECUTED, Price: 7578.91, Cost: -9989.55, Comm 4.07
2018-07-20T16:35:00, Order Canceled/Margin/Rejected
2018-07-20T16:35:00, OPERATION PROFIT, GROSS -176.94, NET -185.01
2018-07-20T17:00:00, S

In [4]:
""" Trade report is provided by Backtrader.analyzers.TradeAnalyzer

"""
def get_win_rate(trade_raport):
    """ Get the win rate from trade report
    
    Win rate = number of win trades / number of total trades
    """
    total_trades = trade_raport['total']['total']
    win_trades = trade_raport['won']['total']
    win_rate = win_trades / total_trades
    return win_rate

def get_roi(trade_raport, init_protfolio_value=100000.0):
    """ Get the ROI from trade report

    ROI = PNL / Initial Portfolio Value
    """
    pnl = trade_raport['pnl']['net']['total']
    roi = pnl / init_protfolio_value
    return roi

def get_drawdown(drawdown_report):
    """ Get the drawdown from drawdown report

    """
    drawdown = drawdown_report['drawdown']
    return drawdown

In [5]:
sessions_win = 0
sessions_win_rate = 0
for i in range(total_sessions):
    trade_report = trade_reports[i]
    drawdown_report = drawdown_reports[i]
    win_rate = get_win_rate(trade_report)
    roi = get_roi(trade_report)
    if roi > 0:
        sessions_win += 1
    drawdown = get_drawdown(drawdown_report)
    print("total_trades: {}, win_rate: {:.2f},  roi: {:.2f}%, drawdown: {:.2f}%.".format(trade_report['total']['total'], win_rate, roi*100, drawdown))
sessions_win_rate = sessions_win/total_sessions * 100
print("sessions_win_rate: {:.2f}%.".format(sessions_win_rate)) 

total_trades: 27, win_rate: 0.59,  roi: -0.72%, drawdown: 0.80%.
sessions_win_rate: 0.00%.
